# [Жёлтое такси в Нью-Йорке](https://www.coursera.org/learn/data-analysis-project/supplement/NuH2K/zhioltoie-taksi-v-n-iu-iorkie)

Нью-Йоркская комиссия по такси и лимузинам (TLC) предоставляет подробные анонимизированные данные о поездках клиентов с 2009 года. Машины, выкрашенные жёлтым, имеют право брать пассажиров на улицах в любом из пяти боро города.

Сырые данные о поездках в жёлтом такси можно найти на сайте TLC: www.nyc.gov/html/tlc/html/about/trip_record_data.shtml Эти данные разбиты на файлы по месяцам. В каждом из файлов содержится следующая информация о поездках:
* VendorID провайдер данных (одна из двух категорий)
* tpep_pickup_datetime - время начала поездки
* tpep_dropoff_datetime - время окончания поездки
* passenger_count - количество пассажиров
* trip_distance - расстояние по счётчику
* pickup_longitude &	pickup_latitude - долгота и широта точки начала поездки
* dropoff_longitude	& dropoff_latitude - долгота и широта точки окончания поездки
* RatecodeID - тип тарифа (одна из шести категорий)
* store_and_fwd_flag	- бинарный флаг, показывающий, были ли данные о поездке получены немедленно после её окончания, или какое-то время хранились в памяти автомобиля
* payment_type - способ оплаты (одна из шести категорий)
* fare_amount - стоимость поездки по счётчику
* extra - доплата за поездки в пиковые часы и ночью
* mta_tax - налог на счётчик
* tip_amount - размер чаевых
* tolls_amount - доплата за проезд по платным дорогам
* improvement_surcharge	- доплата за проезд, взимаемая с каждой поездки с января 2015
* total_amount - общая стоимость поездки

Подробнее: http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

Задача проекта — научиться предсказывать количество поездок в ближайшие часы в каждом районе Нью-Йорка; для простоты мы определим прямоугольные районы. Для того, чтобы её решить, сырые данные необходимо агрегировать по часам и районам. Агрегированные данные будут представлять собой почасовые временные ряды с количествами поездок из каждого района.

Задачу прогнозирования таких рядов можно решать с помощью авторегрессионных моделей, прогнозируя каждый ряд независимо. Ряды явно имеют сложную сезонность — суточную, недельную и годовую, поэтому для их моделирования понадобится использовать модель ARIMA с дополнительной регрессией на внешние признаки.

Чтобы улучшить такую модель, можно попытаться учесть:

* взаимосвязи между рядами
* дополнительные параметры поездок, которые можно посчитать по исходным данным
* внешние календарные и географические признаки

В ходе выполнения проекта вы научитесь:

* работать с геоданными
* прогнозировать временные ряды сложной структуры
* строить и настраивать регрессионные модели, делающие совместные предсказания для большого количества взаимосвязанных рядов

Похожие задачи возникают на практике, если вам необходимо спрогнозировать продажи большого количества товаров в большом количестве магазинов, объём снятия денег в сети банкоматов, посещаемость разных страниц сайта и т.д.

В ходе выполнения проекта у вас будет свобода выбора хода решения, используемых средств анализа, библиотек и моделей. Также обратите внимание, что для его успешного выполнения вам нужно будет скачать и обработать от 10 до 200 гигабайт сырых данных.

In [1]:
%pylab inline

from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats


from itertools import product
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import euclidean
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import warnings

from sklearn.externals import joblib 

Populating the interactive namespace from numpy and matplotlib


C:\Program Files\Conda_3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


---

###### Агрегирование данных в ячейках отобранных на второй неделе за период с января по март 2016

In [3]:
df_region = pd.read_csv('data/df_regions/week4_region.csv', index_col=[0])

N_regions = df_region.index

In [4]:
df_regions = pd.read_csv('data/df_regions/regions.csv', sep=';', index_col=['region'])

NY_longitude = (-74.25559, -73.70001)
NY_latirude = (40.49612, 40.91553)

bin_NY_longitude = list(set(df_regions.west.values))
bin_NY_longitude.append(NY_longitude[1])
bin_NY_longitude.sort()
bin_NY_latirude = list(set(df_regions.south.values))
bin_NY_latirude.append(NY_latirude[1])
bin_NY_latirude.sort()

In [97]:
def gen_periods(date_list, region_n = N_regions):
    '''Агрегирование и сохранение данных из определенного региона'''
    for n in date_list:
        clear_df(pd.read_csv('data/yellow_tripdata/yellow_tripdata_20' + n + '.csv'),
                 region_n,
                 int('20' + n[:2]),
                 int(n[-2:])).to_csv('data/df_regions/week6_df_features_' + n + '.csv')

def clear_df(df, region_n, year_start, month_start, year_end=None, month_end=None, region=df_regions.index):
    '''Агрегирование данных из определенной зоны'''
    df = df[(((df.pickup_longitude > NY_longitude[0]) & (df.pickup_longitude < NY_longitude[1]))
                  & ((df.pickup_latitude > NY_latirude[0]) & (df.pickup_latitude < NY_latirude[1])))]
    
    df.loc[:, 'count'] = 1

    
    ret = stats.binned_statistic_2d(x=df.pickup_longitude.values,
                                    y=df.pickup_latitude.values,
                                    values=df.VendorID,
                                    statistic='count',
                                    bins=[bin_NY_longitude, bin_NY_latirude],
                                    expand_binnumbers=True)
    df['region'] = (ret[3][0] - 1) * 50 + ret[3][1]

    df = df[df['region'].isin(region_n)]

    df.loc[:, 'tpep_pickup_datetime'] = pd.to_datetime(df.loc[:,'tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
    df.loc[:, 'tpep_dropoff_datetime'] = pd.to_datetime(df.loc[:,'tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
    df.loc[:, 'tpep_pickup_datetime_h'] = pd.to_datetime(df.loc[:,'tpep_pickup_datetime'].apply(lambda x: str(x)[0:13] + ":00:00"),
                                                         format='%Y-%m-%d %H:%M:%S')
    
    df = df[((df.tpep_dropoff_datetime - df.tpep_pickup_datetime) > timedelta(0))
            & (df.passenger_count > 0)
            & (df.trip_distance > 0)]

    if year_end==None or month_end==None:
        if month_start > 0 and month_start < 12:
            period = pd.date_range(str(month_start) + '/1/' + str(year_start), str(month_start+1) + '/1/' + str(year_start), freq="1H")[:-1]
        elif month_start == 12:
            period = pd.date_range(str(month_start) + '/1/' + str(year_start), str(1) + '/1/' + str(year_start+1), freq="1H")[:-1]
        else:
            print('Не верный формат данных')
    else:
        period = pd.date_range(str(month_start) + '/1/' + str(year_start), str(month_end) + '/1/' + str(year_end), freq="1H")[:-1]
    
    df['mean_trip_time'] = (df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'])/timedelta(minutes=1)
    
    df = df.loc[:, ['region', 'tpep_pickup_datetime_h', 'passenger_count', 'mean_trip_time', 'trip_distance', 'total_amount']].groupby(['region','tpep_pickup_datetime_h']).mean()
    df.reset_index(inplace=True)  
    
    df.index = df['tpep_pickup_datetime_h']
    df.drop(columns=['tpep_pickup_datetime_h'])
    df_gen = pd.DataFrame(index=period)
    for region in np.unique(np.unique((df.region))):
        df_parse = df.loc[:,['passenger_count', 'mean_trip_time', 'trip_distance', 'total_amount']][df.region == region]
        for column in df_parse.columns:
            df_gen[(str(region), str(column))] = df_parse[str(column)]
    df_gen.columns = pd.MultiIndex.from_tuples(df_gen.columns, names=['region', 'features'])
    df_gen.fillna(0)

    return df_gen

def merge_df(date_list):
    '''Объединение агрегированных данных'''
    df = pd.read_csv('data/df_regions/df_group_' + date_list[0] + '.csv', index_col=[0])
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    for n in date_list[1:]:
        df_merge = pd.read_csv('data/df_regions/df_group_' + n + '.csv', index_col=[0])
        df_merge.index = pd.to_datetime(df_merge.index, format='%Y-%m-%d %H:%M:%S')
        df = pd.concat([df, df_merge])
    return df

In [99]:
%%time
# Агрегирование и сохранение данных для выбранных регионов
temp = gen_periods([
#     '16-01','16-02', '16-03', '16-04', '16-05', '16-06'
    '15-01','15-02','15-03','15-04','15-05','15-06','15-07','15-08','15-09','15-10','15-11','15-12'
], region_n = N_regions)

C:\Program Files\Conda_3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 1h 8min 9s
